In [ ]:
module.exports = {
  networks: {
    development: {
      host: "127.0.0.1",
      port: 8545,
      network_id: "*" // Match any network id
    }
  },
  compilers: {
    solc: {
      version: "0.5.1",
      optimizer: {
        enabled: true,
        runs: 200
      }
    }
  }
};

In [ ]:
// ERC20.sol
pragma solidity ^0.5.0;

interface ERC20 {
    function totalSupply() external view returns (uint256);
    function balanceOf(address account) external view returns (uint256);
    function transfer(address recipient, uint256 amount) external returns (bool);
    function allowance(address owner, address spender) external view returns (uint256);
    function approve(address spender, uint256 amount) external returns (bool);
    function transferFrom(address sender, address recipient, uint256 amount) external returns (bool);

    event Transfer(address indexed from, address indexed to, uint256 value);
    event Approval(address indexed owner, address indexed spender, uint256 value);
}

In [ ]:
mapping(address => uint256) public balances;
mapping(address => mapping(address => uint256)) public allowed;

In [ ]:
// XYZCoin.sol
pragma solidity ^0.5.0;

import "./ERC20.sol";

contract XYZCoin is ERC20 {
    string public name = "XYZCoin";
    string public symbol = "XYZ";
    uint8 public decimals = 0;
    uint256 public totalSupply = 1000;

    mapping(address => uint256) public balances;
    mapping(address => mapping(address => uint256)) public allowed;

    constructor() public {
        balances[msg.sender] = totalSupply;
    }

    function transfer(address recipient, uint256 amount) external override returns (bool) {
        require(balances[msg.sender] >= amount);
        balances[msg.sender] -= amount;
        balances[recipient] += amount;
        emit Transfer(msg.sender, recipient, amount);
        return true;
    }

    function approve(address spender, uint256 amount) external override returns (bool) {
        allowed[msg.sender][spender] = amount;
        emit Approval(msg.sender, spender, amount);
        return true;
    }

    function transferFrom(address sender, address recipient, uint256 amount) external override returns (bool) {
        require(allowed[sender][msg.sender] >= amount);
        require(balances[sender] >= amount);
        allowed[sender][msg.sender] -= amount;
        balances[sender] -= amount;
        balances[recipient] += amount;
        emit Transfer(sender, recipient, amount);
        return true;
    }

    function balanceOf(address account) external view override returns (uint256) {
        return balances[account];
    }

    function totalSupply() external view override returns (uint256) {
        return totalSupply;
    }

    function allowance(address owner, address spender) external view override returns (uint256) {
        return allowed[owner][spender];
    }
}

In [ ]:
// migrations/2_xyz_coin.js
const XYZCoin = artifacts.require("XYZCoin");

module.exports = function(deployer) {
  deployer.deploy(XYZCoin);
};

In [ ]:
const XYZCoin = artifacts.require("XYZCoin");

contract("XYZCoin", async accounts => {
  it("should set the token name correctly", async () => {
    let xyzCoinInstance = await XYZCoin.deployed();
    assert.equal(await xyzCoinInstance.name(), "XYZCoin");
  });
});

In [ ]:
it("should set the total supply correctly", async () => {
  let xyzCoinInstance = await XYZCoin.deployed();
  assert.equal(await xyzCoinInstance.totalSupply(), 1000);
});

it("should allow transfer of tokens", async () => {
  let xyzCoinInstance = await XYZCoin.deployed();
  let initialBalance = await xyzCoinInstance.balanceOf(accounts[0]);
  await xyzCoinInstance.transfer(accounts[1], 10);
  let finalBalance = await xyzCoinInstance.balanceOf(accounts[0]);
  assert.equal(finalBalance, initialBalance - 10);
});

it("should emit Transfer event on transfer", async () => {
  let xyzCoinInstance = await XYZCoin.deployed();
  let receipt = await xyzCoinInstance.transfer(accounts[1], 10);
  assert.equal(receipt.logs.length, 1);
  assert.equal(receipt.logs[0].event, "Transfer");
});

it("should allow approval and transferFrom", async () => {
  let xyzCoinInstance = await XYZCoin.deployed();
  await xyzCoinInstance.approve(accounts[1], 10);
  let receipt = await xyzCoinInstance.transferFrom(accounts[0], accounts[2], 10);
  assert.equal(receipt.logs.length, 1);
  assert.equal(receipt.logs[0].event, "Transfer");
});